# EPC RATING PROBLEM APPROACH
This is a Classification problem and the available features in dataset are mix of numerical and categorical values. Therefore decision tree-based models are good to solve this. Decision trees are capable of handling both types of features without requiring extensive preprocessing, and they are convenient for datasets with mixed feature types.

# Load and clean the data

In [1]:
import pandas as pd
import numpy as np

- Read the column metradata from csv

In [2]:
columns_df = pd.read_csv('data/columns.csv')


In [3]:
columns_df.describe()

,filename,column,datatype,description
count,99,99,99,99
unique,2,98,6,74
top,certificates.csv,LMK_KEY,string,Overall description of the property feature
freq,92,2,64,9


- Extract data types for certificates dataset 

In [4]:
columns_df_cert = columns_df.loc[columns_df['filename'] == 'certificates.csv']

- Extract timestamp related features 

In [5]:
print(columns_df_cert.datatype.unique())

['string' 'integer' 'date' 'float' 'decimal' 'datetime']


- Extract datetime columns 

In [6]:
datetime_columns = columns_df['column'].loc[columns_df['datatype'].isin(['date','datetime'])].to_list()

In [7]:
print(datetime_columns)

['INSPECTION_DATE', 'LODGEMENT_DATE', 'LODGEMENT_DATETIME']


- Change to dtypes suported by pandas

In [8]:
columns_df_cert_updated = columns_df_cert.copy()

In [9]:
replacements = {'string':'str', 'integer':'Int64', 'decimal': 'float'}

In [10]:
columns_df_cert_updated['datatype'] = columns_df_cert['datatype'].map(replacements).fillna(columns_df_cert_updated['datatype'])

- Create dictionary with data types for certificates data set 

In [11]:
dtype_cert = {row['column']:row['datatype'] for index, row in columns_df_cert_updated.iterrows() if row['datatype'] not in ['date', 'datetime']}

- Load certificates dataset

In [12]:
certificates_df = pd.read_csv('data/certificates.csv', dtype=dtype_cert)

- Extract missing data percentage per each column 

In [13]:
missing_data_perc = certificates_df.isnull().sum() / certificates_df.shape[0] * 100

In [14]:
print(missing_data_perc.to_string())

LMK_KEY                            0.000000
ADDRESS1                           0.000000
ADDRESS2                          47.134941
ADDRESS3                          88.717015
POSTCODE                           0.000000
BUILDING_REFERENCE_NUMBER          0.000000
CURRENT_ENERGY_RATING              0.000000
POTENTIAL_ENERGY_RATING            0.000000
CURRENT_ENERGY_EFFICIENCY          0.000000
POTENTIAL_ENERGY_EFFICIENCY        0.000000
PROPERTY_TYPE                      0.000000
BUILT_FORM                         0.023593
INSPECTION_DATE                    0.000000
LOCAL_AUTHORITY                    0.000000
CONSTITUENCY                       0.005445
COUNTY                            27.812126
LODGEMENT_DATE                     0.000000
TRANSACTION_TYPE                   0.000000
ENVIRONMENT_IMPACT_CURRENT         0.000000
ENVIRONMENT_IMPACT_POTENTIAL       0.000000
ENERGY_CONSUMPTION_CURRENT         0.000000
ENERGY_CONSUMPTION_POTENTIAL       0.001210
CO2_EMISSIONS_CURRENT           

- Ignore feature columns with more than 10 % missing data 

In [15]:
large_gaps_data = missing_data_perc.loc[missing_data_perc >10].index.to_list()

In [16]:
data_df = certificates_df.drop(large_gaps_data, axis=1)

- Drop date or datetime columns from data df as these columns are not related to building properties

In [17]:
data_df = data_df.drop(datetime_columns, axis=1)

- Drop columns linked to potential certificate rating

In [18]:
data_df = data_df.drop(data_df.filter(regex='POTENTIAL').columns.to_list(), axis=1)

- Drop high-cardinality categorical variables 

In [19]:
high_cardinality_col = [x for x in data_df.columns if data_df.shape[0]/len(data_df[x].unique()) < 5]

In [20]:
data_df = data_df.drop(high_cardinality_col, axis=1)

- Drop single category columns

In [21]:
single_category_col = [x for x in data_df.columns if len(data_df[x].unique())==1]

In [22]:
data_df = data_df.drop(single_category_col, axis=1)

In [23]:
print(data_df.columns)

Index(['POSTCODE', 'CURRENT_ENERGY_RATING', 'CURRENT_ENERGY_EFFICIENCY',
       'PROPERTY_TYPE', 'BUILT_FORM', 'CONSTITUENCY', 'TRANSACTION_TYPE',
       'ENVIRONMENT_IMPACT_CURRENT', 'ENERGY_CONSUMPTION_CURRENT',
       'CO2_EMISSIONS_CURRENT', 'CO2_EMISS_CURR_PER_FLOOR_AREA',
       'LIGHTING_COST_CURRENT', 'HEATING_COST_CURRENT',
       'HOT_WATER_COST_CURRENT', 'TOTAL_FLOOR_AREA', 'ENERGY_TARIFF',
       'GLAZED_TYPE', 'GLAZED_AREA', 'LOW_ENERGY_LIGHTING',
       'NUMBER_OPEN_FIREPLACES', 'HOTWATER_DESCRIPTION',
       'HOT_WATER_ENERGY_EFF', 'HOT_WATER_ENV_EFF', 'FLOOR_DESCRIPTION',
       'WINDOWS_DESCRIPTION', 'WINDOWS_ENERGY_EFF', 'WINDOWS_ENV_EFF',
       'WALLS_DESCRIPTION', 'WALLS_ENERGY_EFF', 'WALLS_ENV_EFF',
       'ROOF_DESCRIPTION', 'MAINHEAT_DESCRIPTION', 'MAINHEAT_ENERGY_EFF',
       'MAINHEAT_ENV_EFF', 'MAINHEATCONT_DESCRIPTION', 'MAINHEATC_ENERGY_EFF',
       'MAINHEATC_ENV_EFF', 'LIGHTING_DESCRIPTION', 'LIGHTING_ENERGY_EFF',
       'LIGHTING_ENV_EFF', 'MAIN_FUEL', '

- Drop features which are related to the certificates not to building properties 

In [24]:
data_df = data_df.drop(['UPRN_SOURCE', 'TRANSACTION_TYPE'], axis=1)

- Clear the description features in dataset by ignoring special characters

In [25]:
pattern = r'[^a-zA-Z0-9\s+\.]'

In [26]:
data_df['FLOOR_DESCRIPTION'] = data_df['FLOOR_DESCRIPTION'].replace(pattern, '', regex=True)

In [27]:
data_df['HOTWATER_DESCRIPTION'] = data_df['HOTWATER_DESCRIPTION'].replace(pattern, '', regex=True)

In [28]:
data_df['WINDOWS_DESCRIPTION'] = data_df['WINDOWS_DESCRIPTION'].replace(pattern, '', regex=True)

In [29]:
data_df['WALLS_DESCRIPTION'] = data_df['WALLS_DESCRIPTION'].replace(pattern, '', regex=True)

In [30]:
data_df['ROOF_DESCRIPTION'] = data_df['ROOF_DESCRIPTION'].replace(pattern, '', regex=True)

In [31]:
data_df['MAINHEAT_DESCRIPTION'] = data_df['MAINHEAT_DESCRIPTION'].replace(pattern, '', regex=True)

In [32]:
data_df['LIGHTING_DESCRIPTION'] = data_df['LIGHTING_DESCRIPTION'].replace(pattern, '', regex=True)

In [33]:
print(data_df.columns)

Index(['POSTCODE', 'CURRENT_ENERGY_RATING', 'CURRENT_ENERGY_EFFICIENCY',
       'PROPERTY_TYPE', 'BUILT_FORM', 'CONSTITUENCY',
       'ENVIRONMENT_IMPACT_CURRENT', 'ENERGY_CONSUMPTION_CURRENT',
       'CO2_EMISSIONS_CURRENT', 'CO2_EMISS_CURR_PER_FLOOR_AREA',
       'LIGHTING_COST_CURRENT', 'HEATING_COST_CURRENT',
       'HOT_WATER_COST_CURRENT', 'TOTAL_FLOOR_AREA', 'ENERGY_TARIFF',
       'GLAZED_TYPE', 'GLAZED_AREA', 'LOW_ENERGY_LIGHTING',
       'NUMBER_OPEN_FIREPLACES', 'HOTWATER_DESCRIPTION',
       'HOT_WATER_ENERGY_EFF', 'HOT_WATER_ENV_EFF', 'FLOOR_DESCRIPTION',
       'WINDOWS_DESCRIPTION', 'WINDOWS_ENERGY_EFF', 'WINDOWS_ENV_EFF',
       'WALLS_DESCRIPTION', 'WALLS_ENERGY_EFF', 'WALLS_ENV_EFF',
       'ROOF_DESCRIPTION', 'MAINHEAT_DESCRIPTION', 'MAINHEAT_ENERGY_EFF',
       'MAINHEAT_ENV_EFF', 'MAINHEATCONT_DESCRIPTION', 'MAINHEATC_ENERGY_EFF',
       'MAINHEATC_ENV_EFF', 'LIGHTING_DESCRIPTION', 'LIGHTING_ENERGY_EFF',
       'LIGHTING_ENV_EFF', 'MAIN_FUEL', 'WIND_TURBINE_COUNT',

# Building and training of the prediction model

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

- Drop nan valueas ac gbc model does not accept it 

In [35]:
data_df = data_df.dropna()

- Choose featureas as building geometry location and characteristics that could be supplied from different sources than EPC

In [36]:
features = data_df[["POSTCODE", "PROPERTY_TYPE", "CONSTITUENCY", "ENERGY_CONSUMPTION_CURRENT",
                    "LIGHTING_COST_CURRENT", "HEATING_COST_CURRENT", "TOTAL_FLOOR_AREA", 
                    "HOT_WATER_COST_CURRENT", "ENERGY_TARIFF", "GLAZED_TYPE", "GLAZED_AREA",
                    "LOW_ENERGY_LIGHTING", "NUMBER_OPEN_FIREPLACES", "HOTWATER_DESCRIPTION",
                    "FLOOR_DESCRIPTION","WINDOWS_DESCRIPTION", "WALLS_DESCRIPTION", "ROOF_DESCRIPTION", 
                    "MAINHEAT_DESCRIPTION", "MAINHEATCONT_DESCRIPTION", "LIGHTING_DESCRIPTION",
                    "MAIN_FUEL", "WIND_TURBINE_COUNT", "MECHANICAL_VENTILATION",
                    "CONSTITUENCY_LABEL", "POSTTOWN", "CONSTRUCTION_AGE_BAND", "TENURE"]]

In [37]:
target = data_df['CURRENT_ENERGY_RATING'].to_list()

- Split data to training and test data sets

In [38]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, stratify=target)

- Split features for numerical and categorical

In [39]:
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.to_list()
categorical_features = X_train.select_dtypes(include=['object']).columns.to_list()

- Define categorical features transformer to turn strings to numerical data

In [40]:
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

- Define numerical features transformer to rescale variables into the range [0,1]

In [41]:
numerical_transformer = Pipeline(steps=[
    ('minmax', MinMaxScaler())])

- Apply transformers to numerical and categorical features

In [42]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', categorical_transformer, categorical_features)])

- Define Gradient Boosting Classifier

In [43]:
gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)

- Create a pipeline combining preprocessing and model

In [44]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', gb_classifier)])


- Train the model

In [45]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['ENERGY_CONSUMPTION_CURRENT',
                                                   'LIGHTING_COST_CURRENT',
                                                   'HEATING_COST_CURRENT',
                                                   'TOTAL_FLOOR_AREA',
                                                   'HOT_WATER_COST_CURRENT',
                                                   'LOW_ENERGY_LIGHTING',
                                                   'NUMBER_OPEN_FIREPLACES',
                                                   'WIND_TURBINE_COUNT']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))])...
                                                   'GLAZED_TYPE', 'GLAZED_AREA',
                                                   'HOTWATER_DESCRIPTION',
                                                   'FLOOR_DESCRIPTION',
                                                   'WINDOWS_DESCRIPTION',
                                                   'WALLS_DESCRIPTION',
                                                   'ROOF_DESCRIPTION',
                                                   'MAINHEAT_DESCRIPTION',
                                                   'MAINHEATCONT_DESCRIPTION',
                                                   'LIGHTING_DESCRIPTION',
                                                   'MAIN_FUEL',
                                                   'MECHANICAL_VENTILATION',
                                                   'CONSTITUENCY_LABEL',
                                                   'POSTTOWN',
                                                   'CONSTRUCTION_AGE_BAND',
                                                   'TENURE'])])),
                ('classifier', GradientBoostingClassifier(random_state=42))])

- Make a prediction for test data

In [47]:
y_predict=pipeline.predict(X_test)

- Evaluate predictions

In [48]:
accuracy = accuracy_score(y_test, y_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 88.69%
